In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch

from exercise_code.data.segmentation_dataset import SegmentationData, label_img_to_rgb
from exercise_code.data.download_utils import download_dataset
from exercise_code.util import visualizer, save_model
from exercise_code.util.Util import checkSize, checkParams, test
from exercise_code.networks.segmentation_nn import SegmentationNN, DummySegmentationModel
from exercise_code.tests import test_seg_nn
#set up default cuda device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
model = torch.load('finalmodel.pt')
model.eval()
model = model.to('cuda') 

In [3]:
import os

import numpy as np
import torch
import torch.utils.data as data
from PIL import Image
from torchvision import transforms

In [4]:
def output(model,filename):
    img = Image.open(filename).convert('RGB')
    w, h = img.size
    to_tensor = transforms.ToTensor()
    small_size = transforms.Resize((256,256))
    orig_size = transforms.Resize((h,w))
    
    img = small_size(img)
    img = to_tensor(img).to('cuda')
    img = img.unsqueeze(0)
    outputs = model.forward(img)
    _, preds = torch.max(outputs, 1)
    pred = preds.data.cpu().numpy()
    pred = label_img_to_rgb(pred)
    pred=Image.fromarray(pred)
    pred = orig_size(pred)
    pred.save(filename[:-3]+'png')
    #pred = orig_size(pred)
    


In [8]:
filelist = []
for i in range(200):
    filelist.append('Testing/'+str(i)+'.jpg')
for i in filelist:
    output(model,i)